In [33]:
class multimethod:
    def __init__(self, dispatcher):
        self._dispatch = dispatcher
        self._methods = []

    def __call__(self, *args, **kwargs):
        dispatch = self._dispatch(*args, **kwargs)
        for (specs, kwspecs, method) in self._methods:
            if dispatch(*specs, **kwspecs):
                return method(*args, **kwargs)

    def add_method(self, specs, kwspecs,  method):
        self._methods.append((specs, kwspecs, method))

def method(generic, *specs, **kwspecs):
    def decorator(method):
        generic.add_method(specs, kwspecs, method)
        return generic
    return decorator

def type_dispatch(*args, **kwargs):
    def dispatcher(*targs, **kwtargs):
        posdispatch = all(map(isinstance, args, targs))
        kwdispatch = all(isinstance(x,kwtargs[kx]) for (kx,x) in kwargs.items())
        return posdispatch and kwdispatch
    return dispatcher

def key_dispatch(key, *args, **kwargs):
    def dispatcher(*values, **kwvalues):
        posdispatch = all(arg[key] == value for (arg, value) in zip(args, values))
        kwdispatch = all(arg[key] == kwvalues[karg] for (karg, arg) in kwargs.items())
        return posdispatch and kwdispatch
    return dispatcher
    

In [28]:
@multimethod
def add(x,y):
    return type_dispatch(x,y)

In [29]:
@method(add, int, int)
def add(x,y):
    return x + y

In [30]:
add(1,2)

3

In [31]:
@method(add, int, str)
def add(x,y):
    return x+int(y)

In [32]:
add(1,"10")

11

In [35]:
@multimethod
def say(x, y):
    return key_dispatch("type", x, y)

In [37]:
@method(say, "person", "person")
def say(x,y):
    return "I say '{}', you say '{}'".format(x.get("what"), y.get("what"))

In [38]:
say({"type": "person", "what": "Hello!"}, {"type": "person", "what": "goodbye!"})

"I say 'Hello!', you say 'goodbye!'"

In [39]:
@method(say, "person", "robot")
def say(x,y):
    return "I say '{}', you say 'bip boop {} bip boop'".format(x.get("what"), y.get("what"))

In [41]:
say({"type": "person", "what": "Hello!"}, {"type": "robot", "what": "GOODBYE!"})

"I say 'Hello!', you say 'bip boop GOODBYE! bip boop'"